In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('/home/hrai/codes/hpe_library/')
from lib_import import *
from my_utils import *
from sympy import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
th4 = symbols('theta_4')
tg = symbols('tg')
l4 = symbols('l_i')
tg, l4

(tg, l_i)

In [ ]:
def IK_for_limb(dx, dy, dz, l4, l5)
    tg = sqrt(dx**2 + dy**2 + dz**2)
    th4 = acos((tg**2 - l4**2 - l5**2)/(2*l4*l5))
    if dz == 0:
        th1 = -np.pi/2 - phi